## Netflix DATASET Analysing

---

Luis Angel Garcia (2230177)

## Libraries

In [54]:
import configparser
import pandas as pd
import mysql.connector

## MySql Config And Connection

In [55]:
config = configparser.ConfigParser()
config.read("config.txt")
db_config = {
    "host": config.get("DEFAULT", "host"),
    "port": config.getint("DEFAULT", "port"),
    "user": config.get("DEFAULT", "user"),
    "password": config.get("DEFAULT", "password")
}

conn = mysql.connector.connect(**db_config)
cursor= conn.cursor()

## Creacion de DATABASE y Conexion

In [56]:
database_name = config.get("DEFAULT", "database")
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
conn.commit()

conn.database = database_name
print(f"Base de datos '{database_name}' lista para usarse.")

Base de datos 'candidates_database' lista para usarse.


## Creacion de tabla de datos sucios

In [57]:
cursor.execute(""" 
CREATE TABLE IF NOT EXISTS candidates_original (
    id INT AUTO_INCREMENT PRIMARY KEY,
    First_name VARCHAR(20),
    last_name VARCHAR(20),
    email VARCHAR(255) ,
    application_date DATE,
    country VARCHAR(100),
    yoe INT,
    seniority VARCHAR(50),
    technology VARCHAR(50),
    code_challenge_score INT,
    technical_interview_score INT
);
""")
conn.commit()
print ("La base de datos y tabla se ha creado con exito")

La base de datos y tabla se ha creado con exito


## Carga de datos

In [58]:
csv_file= './DATASET/candidates.csv'
df = pd.read_csv(csv_file, sep=";")
df.columns = ["first_name", "last_name", "email", "application_date", "country", "yoe", "seniority", "technology", "code_challenge_score", "technical_interview_score"]

df = df.fillna("TEMP_NULL") 

cursor.execute("SELECT COUNT(*) FROM candidates_original")
resultado = cursor.fetchone()
if resultado[0] == 0:
    for _, row in df.iterrows():
        cursor.execute("""
                   INSERT INTO candidates_original (first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score)
                   VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
                """, (row['first_name'],row['last_name'],row['email'],row['application_date'],row['country'],row['yoe'],row['seniority'],row['technology'],row['code_challenge_score'],row['technical_interview_score']))
    conn.commit()
    print("Dato cargados correcctamente")
else:
    conn.commit()
    print("No es necesario volver a cargar los datos")

No es necesario volver a cargar los datos


## use dataset from mysql

In [59]:
cursor.execute("SELECT * FROM candidates_original")

columns = [col[0] for col in cursor.description]
datas= cursor.fetchall()
df = pd.DataFrame(datas, columns=columns)

## Limpieza de datos duplicados y nulos

In [60]:
df_cleaned = df.dropna()
df_cleaned = df_cleaned.drop_duplicates()

## Conversion de application date como formato fecha

In [61]:
df_cleaned['application_date'] = pd.to_datetime(df['application_date'], errors='coerce')

## Eliminacion de filas de fecha invalidad

In [62]:
df_cleaned = df_cleaned.dropna(subset=['application_date'])

## Creacion de columna

In [63]:
df_cleaned['hired'] = ((df_cleaned['code_challenge_score'] >= 7) & (df['technical_interview_score'] >= 7)).astype(int)
print(df_cleaned.columns)

Index(['id', 'First_name', 'last_name', 'email', 'application_date', 'country',
       'yoe', 'seniority', 'technology', 'code_challenge_score',
       'technical_interview_score', 'hired'],
      dtype='object')


## Registro de contratados

In [64]:
df_hired = df_cleaned[df_cleaned['hired'] == 1]

## creacion de tabla de datos limpia

In [66]:
cursor.execute(""" 
CREATE TABLE IF NOT EXISTS candidates_cleaned (
    id INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(20),
    last_name VARCHAR(20),
    email VARCHAR(255) ,
    application_date DATE,
    country VARCHAR(100),
    yoe INT,
    seniority VARCHAR(50),
    technology VARCHAR(50),
    code_challenge_score INT,
    technical_interview_score INT,
    hired BOOLEAN);
""")
conn.commit()
print ("La tabla se ha creado con exito")

La tabla se ha creado con exito


## Carga de datos limpios

In [67]:
cursor.execute("SELECT COUNT(*) FROM candidates_cleaned")
resultado = cursor.fetchone()
if resultado[0] == 0:
    for _, row in df_cleaned.iterrows():
        cursor.execute("""
                   INSERT INTO candidates_cleaned (first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,hired)
                   VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
                """, (row['first_name'],row['last_name'],row['email'],row['application_date'],row['country'],row['yoe'],row['seniority'],row['technology'],row['code_challenge_score'],row['technical_interview_score'],row['hired']))
    conn.commit()
    print("Dato cargados correcctamente")
else:
    conn.commit()
    print("No es necesario volver a cargar los datos")

KeyError: 'first_name'

In [37]:
cursor.close()
conn.close()